In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import site

face_cascade = cv2.CascadeClassifier('c:/Users/Julia Yilmaz/AppData/Local/Programs/Python/Python311/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier('c:/Users/Julia Yilmaz/AppData/Local/Programs/Python/Python311/Lib/site-packages/cv2/data/haarcascade_eye.xml')


In [29]:
# Variables
PREVIEW   = 0   # Preview Mode
GRAY      = 1   # Grayscale Filter
THRESHOLD = 2   # Threshold Filter
SHARPEN = 3 # Sharpen Filter
BLUR = 4 # Blur Filter
SOBEL = 5 # Sobel Filter
HSV = 6 # HSV
OBJECTDETECT = 7 #OBJECT Detection

In [30]:
# Threshold Filter
def Threshold_filter(frame):

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    h = frame.shape[0]
    w = frame.shape[1]

    imgThres = np.zeros((frame.shape[0],frame.shape[1]), dtype=np.uint8)

    for y in range(0,h):
    #Search each pixel in the row
        for x in range(0,w):
        
            #If the pixel value is above 100 set it to white.  If it is below 100 set it to black.
            if (frame[y,x] > 100):
                imgThres[y,x] = 255
            else:
                imgThres[y,x] = 0
    
    
    return imgThres

In [31]:
# Gaussian Filter
def Gaussian_Filter(frame):
    gaussian = np.array([[1,2,1],[2,4,2],[1,2,1]], dtype=np.float32) / 16
    imgGaussian = cv2.filter2D(frame, -1, gaussian)
    return imgGaussian

In [32]:
# Sharpen Filter 
def Sharpen_Filter(frame):
    sharpKernel = np.array([[0,-1, 0], [-1 ,5,-1] , [0,-1,0 ]], dtype=np.float32) 
    imgSharp = cv2.filter2D(frame, -1, sharpKernel)
    return imgSharp

In [33]:
# Sobel Filter
def Sobel_Filter(frame):
    
    gaussian = Gaussian_Filter(frame)

    sobelx = cv2.Sobel(gaussian, cv2.CV_64F, 1, 0, ksize=3) 
    sobely = cv2.Sobel(gaussian, cv2.CV_64F, 0, 1, ksize=3) 

    mag = cv2.magnitude(sobelx, sobely)
    
    th, imgSobel = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY);
    return imgSobel

In [34]:
def Hsv_Filter(frame):
    min = np.array([0,45,0], np.uint8)
    max = np.array([135,135,220], np.uint8)

    skinArea = cv2.inRange(frame, min, max)
    skinHSV = cv2.bitwise_and(frame, frame, mask=skinArea)

    return skinHSV

In [35]:
def Object_Detection(frame):

    # Load the input image and convert it to grayscale

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        center = (round(x + w/2), round(y + h/2))  
        radius = int(round((w + h)*0.25))  
        frame = cv2.circle(frame, center, radius, (255, 0, 0), 4)  
        faceROI = gray[y:y+h,x:x+w] 
        eyes = eyes_cascade.detectMultiScale(faceROI)
 
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)  
            radius = int(round((w2 + h2)*0.25)) 
            frame = cv2.circle(frame, eye_center, radius, (0, 0, 255), 4)  

    ObjectDetect = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return ObjectDetect

In [36]:
# Video Image Loop

# Create a VideoCapture object and read from input file
# If the input is taken from the camera, pass 0 instead of the video file name.
cap = cv2.VideoCapture(0)

# Variable to keep track of the current image filter. Default set to Preview.
image_filter = PREVIEW

# Video Loop
while cap.isOpened():
    # Ret = True if frame is read correctly, frame = the frame that was read.
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        break
    
    # Resize the image so your app runs faster.  Less Pixels to Process.  
    # Use Nearest Neighbor Interpolation for the fastest runtime.
    frame = cv2.resize(frame, None, fx = 0.3, fy = 0.3, interpolation=cv2.INTER_NEAREST)
    
    # Send your frame to each filter.
    if image_filter == PREVIEW:
        # No filter.
        result = frame
    elif image_filter == THRESHOLD:
        # Send the frame to the Threshold function.
        result = Threshold_filter(frame)
    elif image_filter == GRAY:
        # Convert to grayscale.
        result = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif image_filter == SHARPEN:
        # Sharpen Image.
        result = Sharpen_Filter(frame)
    elif image_filter == BLUR:
        # Blur Image.
        result = Gaussian_Filter(frame)
    elif image_filter == SOBEL:
        #Sobel Filter
        result = Sobel_Filter(frame)
    elif image_filter == HSV:
        # HSV masking
        result = Hsv_Filter(frame)
    elif image_filter == OBJECTDETECT:
        result = Object_Detection(frame)
  

  # Show your frame.  Remember you don't need to convert to RGB.  
    cv2.imshow('frame', result)
    
    # Map all your key codes to your filters/trackers. Capital and lower case.
    
    key = cv2.waitKey(1)
     # Quit the app.
    if key == ord('Q') or key == ord('q'):
        break
    # Grayscale filter
    elif key == ord('G') or key == ord('g'):
        image_filter = GRAY
    # Threshold filter
    elif key == ord('T') or key == ord('t'):
        image_filter = THRESHOLD
    # Preview. No filter.
    elif key == ord('P') or key == ord('p'):
        image_filter = PREVIEW
    # Sharpen Image.
    elif key == ord('S') or key == ord('s'):
        image_filter = SHARPEN
    # Blur Image
    elif key == ord('B') or key == ord('b'):
        image_filter = BLUR
    # Sobel Filter
    elif key == ord('E') or key == ord('e'):
        image_filter = SOBEL
    # Mask Filter
    elif key == ord('M') or key == ord('m'):
        image_filter = HSV
    # Object Detection 
    elif key == ord('O') or key == ord('o'):
        image_filter = OBJECTDETECT




##################################################################################
# Close the app
cap.release()
cv2.destroyWindow('frame')   
